## Preambule

In [1]:
# Plotting
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import xarray as xr
from matplotlib import colors
from plotly.subplots import make_subplots

## Input

In [2]:
rules = ["GF", "PC", "PCC", "AP", "GDR", "ECPC"]
path_data = "K:/Data/Data_effortsharing/DataUpdate_ongoing/"

## Read data

In [3]:
xr_dataread = xr.open_dataset(path_data + "startyear_2021/xr_dataread.nc")
xr_2030 = xr.open_dataset(path_data + "startyear_2021/xr_alloc_2030.nc")
xr_2040 = xr.open_dataset(path_data + "startyear_2021/xr_alloc_2040.nc")

all_regions_iso = np.load(path_data + "all_regions.npy")
all_regions_names = np.load(path_data + "all_regions_names.npy")
all_countries_iso = np.load(path_data + "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data + "all_countries_names.npy", allow_pickle=True)

In [4]:
df_eng = pd.read_csv(
    "X:/user/dekkerm/Data/ENGAGE/PolicyScenarios/engage-internal_snapshot_1695642798.csv"
)
df_eng = df_eng[df_eng.Variable == "Emissions|CO2"]
df_eng = df_eng.reset_index(drop=True)
regions_df = np.array(df_eng.Region)
regions_df[regions_df == "Argentine Republic"] = "ARG"
regions_df[regions_df == "Canada"] = "CAN"
regions_df[regions_df == "Commonwealth of Australia"] = "AUS"
regions_df[regions_df == "Federative Republic of Brazil"] = "BRA"
regions_df[regions_df == "People's Repulic of China"] = "CHN"
regions_df[regions_df == "European Union (28 member countries)"] = "EU"
regions_df[regions_df == "Republic of India"] = "IND"
regions_df[regions_df == "Republic of Indonesia"] = "IDN"
regions_df[regions_df == "State of Japan"] = "JPN"
regions_df[regions_df == "Russian Federation"] = "RUS"
regions_df[regions_df == "Kingdom of Saudi Arabia"] = "SAU"
regions_df[regions_df == "Republic of South Africa"] = "ZAF"
regions_df[regions_df == "Republic of Korea (South Korea)"] = "KOR"
regions_df[regions_df == "United Mexican States"] = "MEX"
regions_df[regions_df == "Republic of Turkey"] = "TUR"
regions_df[regions_df == "United States of America"] = "USA"
regions_df[regions_df == "Viet Nam "] = "VNM"
df_eng.Region = regions_df
df = pd.read_excel(
    "X:/user/dekkerm/Data//UNFCCC_Parties_Groups_noeu.xlsx", sheet_name="Country groups"
)
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [5]:
curpol = "GP_CurPol_T45"
ndc = "GP_NDC2030_T45"
nz = "GP_Glasgow"
df_eng_ref = df_eng[["Model", "Scenario", "Region"] + list(df_eng.keys()[5:])]
df_eng_ref = df_eng_ref[df_eng_ref.Scenario.isin([curpol, ndc, nz])]
scen = np.array(df_eng_ref.Scenario)
scen[scen == ndc] = "NDC"
scen[scen == curpol] = "CurPol"
scen[scen == nz] = "NetZero"
reg = np.array(df_eng_ref.Region)
reg[reg == "World"] = "WORLD"
df_eng_ref["Scenario"] = scen
df_eng_ref["Region"] = reg
dummy = df_eng_ref.melt(
    id_vars=["Scenario", "Model", "Region"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Scenario", "Model", "Region", "Time"])
xr_eng = xr.Dataset.from_dataframe(dummy)
xr_eng = xr_eng.reindex(Time=np.arange(1850, 2101))
xr_eng = xr_eng.interpolate_na(dim="Time", method="linear")

Cost optimal

In [6]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(["Emissions|CO2", "Policy Cost|Consumption Loss"])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel(
    "X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",
    sheet_name="meta_Ch3vetted_withclimate",
)
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6_meta["ModelScenario"] = modscens_meta
df_ar6_meta = df_ar6_meta[["ModelScenario", "Category", "Policy_category"]]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(["C1", "C2"])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6["ModelScenario"] = modscens
df_ar6 = df_ar6.drop(["Model", "Scenario", "Unit"], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(
    id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time=np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

In [7]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(["Emissions|CO2", "Policy Cost|Consumption Loss"])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel(
    "X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",
    sheet_name="meta_Ch3vetted_withclimate",
)
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6_meta["ModelScenario"] = modscens_meta
df_ar6_meta = df_ar6_meta[["ModelScenario", "Category", "Policy_category"]]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(["C3"])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6["ModelScenario"] = modscens
df_ar6 = df_ar6.drop(["Model", "Scenario", "Unit"], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(
    id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r2 = xr.Dataset.from_dataframe(dummy)
xr_scen_r2 = xr_scen_r2.reindex(Time=np.arange(1850, 2101))
xr_scen_r2 = xr_scen_r2.interpolate_na(dim="Time", method="linear")

In [8]:
def create_rgb(col):
    return "rgba(" + str(col[0]) + "," + str(col[1]) + "," + str(col[2]) + "," + str(col[3]) + ")"

## Prep data

In [23]:
rulecolors = [
    "sienna",
    "goldenrod",
    "mediumvioletred",
    "forestgreen",
    "tomato",
    "steelblue",
    "silver",
]
rulecolors_rgb = np.array([colors.to_rgba(i) for i in rulecolors]) * 256
for i in range(len(rulecolors_rgb)):
    rulecolors_rgb[i, -1] = 0.05
rulecolors_rgb_s = [create_rgb(i) for i in rulecolors_rgb]
rules = ["GF", "PCC", "PC", "AP", "ECPC", "GDR"]
rules_nodisc = ["GF", "PCC", "AP", "ECPC", "GDR"]
rulecolors_nodisc = [
    "sienna",
    "goldenrod",
    "mediumvioletred",
    "steelblue",
    "tomato",
]  # ['sienna', 'goldenrod', 'mediumvioletred', 'steelblue', 'tomato']
maps_nodisc = ["turbid", "YlOrBr", "PuRd", "Blues", "Reds"]
standard_settings = {
    "NonCO2red": 0.5,
    "Timing": "Immediate",
    "NegEmis": 0.5,
    "Convergence_year": 2050,
    "Discount_factor": 0,
    "Historical_startyear": 1990,
    "Capability_threshold": "Th",
    "RCI_weight": "Half",
}
rulenames = [
    r"Continuity (Grandfathering)",
    r"Continuity & Equality (Per capita convergence)",
    r"Equality, discontinuous (Immediate per capita)",
    r"Capability (Ability to pay)",
    r"Equality & Responsibility (Equal-cumulative per capita)",
    r"Capability & Responsibility (Greenhouse development rights)",
    r"Equality (Per capita budget linear)",
]

In [10]:
def compute_traces_a(REGION, xr_cty):
    traces_a = []
    for cty in all_countries_iso:
        xrreg = xr_med.sel(Region=cty)
        medians = np.array([xrreg[i] for i in rules_nodisc])
        idx = np.argmax(medians)
        if cty == REGION:
            traces_a.append(
                go.Choropleth(
                    locations=[cty],
                    z=[0.8],
                    locationmode="ISO-3",
                    colorscale=maps_nodisc[idx],
                    zmax=1.5,
                    zmin=0.0,
                    hovertemplate="%{text}",
                    name="",
                    marker_line_color="black",
                    marker_line_width=3,
                    showscale=False,
                )
            )
        else:
            traces_a.append(
                go.Choropleth(
                    locations=[cty],
                    z=[0.8] * len(ctys),
                    locationmode="ISO-3",
                    colorscale=maps_nodisc[idx],
                    zmax=1.5,
                    zmin=0.0,
                    hovertemplate="%{text}",
                    name="",
                    marker_line_color="white",
                    marker_line_width=0.5,
                    showscale=False,
                    marker={"opacity": 0.5},
                )
            )
    xrreg = xr_med.sel(Region=REGION)
    medians = np.array([xrreg[i] for i in rules_nodisc])
    idx = np.argmax(medians)
    traces_a.append(
        go.Choropleth(
            locations=[REGION],
            z=[0.8],
            locationmode="ISO-3",
            colorscale=maps_nodisc[idx],
            zmax=1.5,
            zmin=0.0,
            hovertemplate="%{text}",
            name="",
            marker_line_color="black",
            marker_line_width=3,
            showscale=False,
        )
    )
    return traces_a

In [26]:
def compute_traces_b(REGION, xr_cty):
    traces_b = []

    # Zero line
    traces_b.append(
        go.Scatter(
            x=np.arange(1990, 2101),
            y=[0] * len(np.arange(1990, 2101)),
            name="-",
            line={"color": "black", "width": 3, "dash": "dot"},
            mode="lines",
            fill=None,
            showlegend=False,
        )
    )

    # 1990 line
    # traces_b.append(go.Scatter(x=np.arange(1990, 2101),
    #                            y=[xr_dataread.sel(Time=1990, Region=ctys).GHG_hist.sum(dim='Region')]*len(np.arange(1990, 2101)),
    #                            name="-",
    #                            line={'color': 'black', 'width': 1, 'dash': 'dot'},
    #                            mode='lines',
    #                            fill=None,
    #                            showlegend=False))

    # 90% reduction line
    traces_b.append(
        go.Scatter(
            x=np.arange(1990, 2101),
            y=[0.1 * xr_dataread.sel(Time=1990, Region=ctys).GHG_hist.sum(dim="Region")]
            * len(np.arange(1990, 2101)),
            name="-",
            line={"color": "black", "width": 1, "dash": "dot"},
            mode="lines",
            fill=None,
            showlegend=False,
        )
    )
    traces_b.append(
        go.Scatter(
            x=[2012],
            y=[0.1 * xr_dataread.sel(Time=1990, Region=ctys).GHG_hist.sum(dim="Region")],
            name="-",
            line={"color": "black", "width": 1, "dash": "dot"},
            mode="text",
            textposition="top right",
            text="90% reduction w.r.t. 1990 emissions",
            fill=None,
            showlegend=False,
        )
    )

    # 2040 line
    traces_b.append(
        go.Scatter(
            x=[2040, 2040],
            y=[-1e5, 1e5],
            name="-",
            line={"color": "black", "width": 1, "dash": "dot"},
            mode="lines",
            fill=None,
            showlegend=False,
        )
    )

    # Historical emissions
    traces_b.append(
        go.Scatter(
            x=np.arange(1990, 2022),
            y=xr_dataread.sel(Time=np.arange(1990, 2022), Region=ctys).GHG_hist.sum(dim="Region"),
            name="-",
            line={"color": "black", "width": 7},
            mode="lines",
            fill=None,
            showlegend=False,
        )
    )

    # Allocations
    # Median
    past = float(xr_dataread.sel(Time=1990, Region=ctys).GHG_hist.sum(dim="Region"))
    for r_i, r in enumerate(rules):
        traces_b.append(
            go.Scatter(
                x=np.arange(2021, 2041),
                y=xr_cty.sel(
                    Time=np.arange(2021, 2041), Temperature=1.6, Risk=0.5, **standard_settings
                ).mean(dim="Scenario")[r],
                name=rulenames[r_i],
                line={
                    "color": rulecolors[r_i],
                    "width": 7,
                    "dash": ["dot", "dot", "dot", "solid", "solid", "solid", "solid"][r_i],
                },
                mode="lines",
                fill=None,
                showlegend=True,
            )
        )
        val = float(
            xr_cty.sel(Time=2040, Temperature=1.6, Risk=0.5, **standard_settings).mean(
                dim="Scenario"
            )[r]
        )
        traces_b.append(
            go.Scatter(
                x=[2040],
                y=[val],
                name=rulenames[r_i],
                text=[str(np.round((past - val) / past * 100, 1)) + "%"],
                line={"color": rulecolors[r_i], "width": 7},
                mode="text",
                textposition="middle right",
                textfont={"size": 20, "color": "black"},
                fill=None,
                showlegend=False,
            )
        )

    # Full spread (min/max)
    for r_i, r in enumerate(rules):
        traces_b.append(
            go.Scatter(
                x=np.arange(2021, 2101),
                y=xr_cty.sel(Time=np.arange(2021, 2101), Temperature=1.6, Risk=0.5)
                .mean(dim="Scenario")
                .min(dim=standard_settings.keys())[r],
                name="",
                line={"color": rulecolors[r_i], "width": 0},
                fillcolor=rulecolors_rgb_s[r_i],
                mode="lines",
                fill=None,
                showlegend=False,
            )
        )
        traces_b.append(
            go.Scatter(
                x=np.arange(2021, 2101),
                y=xr_cty.sel(Time=np.arange(2021, 2101), Temperature=1.6, Risk=0.5)
                .mean(dim="Scenario")
                .max(dim=standard_settings.keys())[r],
                name="",
                line={"color": rulecolors[r_i], "width": 0},
                fillcolor=rulecolors_rgb_s[r_i],
                mode="lines",
                fill="tonexty",
                showlegend=False,
            )
        )
    return traces_b

In [12]:
def compute_traces_c(REGION, xr_cty):
    traces_c = []
    for cty_i, cty in enumerate(all_countries_iso):
        pop = float(xr_dataread.Population.sel(Scenario="SSP2", Region=cty, Time=2021))
        if not np.isnan(pop):
            if pop > 1e-1:
                popn = np.sqrt(
                    pop
                    / np.nanmax(
                        xr_dataread.Population.sel(
                            Scenario="SSP2", Region=all_countries_iso, Time=2021
                        )
                    )
                )
                if cty == REGION:
                    traces_c.append(
                        go.Scatter(
                            x=[
                                xr_dataread.GDP.sel(Scenario="SSP2", Region=cty, Time=2021)
                                * 1e9
                                / pop
                                / 1e6
                            ],
                            y=[
                                xr_dataread.GHG_hist.sel(
                                    Time=np.arange(1950, 2021), Region=cty
                                ).sum(dim="Time")
                                / pop
                                / 1e6
                                * 1e3
                            ],
                            mode="markers+text",
                            text=cty,
                            textfont={"size": popn * 32 + 3, "color": "black"},
                            marker={
                                "size": popn * 60 + 8,
                                "color": cols_ctys2[cty_i],
                                "opacity": 0.7,
                                "line": {"color": "black", "width": 10},
                            },
                            showlegend=False,
                        )
                    )
                else:
                    traces_c.append(
                        go.Scatter(
                            x=[
                                xr_dataread.GDP.sel(Scenario="SSP2", Region=cty, Time=2021)
                                * 1e9
                                / pop
                                / 1e6
                            ],
                            y=[
                                xr_dataread.GHG_hist.sel(
                                    Time=np.arange(1950, 2021), Region=cty
                                ).sum(dim="Time")
                                / pop
                                / 1e6
                                * 1e3
                            ],
                            mode="markers+text",
                            text=cty,
                            textfont={"size": popn * 32 + 3, "color": "black"},
                            marker={
                                "size": popn * 60 + 8,
                                "color": cols_ctys2[cty_i],
                                "opacity": 0.7,
                            },
                            showlegend=False,
                        )
                    )

    for cty_i, cty in enumerate(all_countries_iso):
        if cty == REGION:
            pop = float(xr_dataread.Population.sel(Scenario="SSP2", Region=cty, Time=2021))
            if not np.isnan(pop):
                if pop > 1e-1:
                    popn = np.sqrt(
                        pop
                        / np.nanmax(
                            xr_dataread.Population.sel(
                                Scenario="SSP2", Region=all_countries_iso, Time=2021
                            )
                        )
                    )
                    traces_c.append(
                        go.Scatter(
                            x=[
                                xr_dataread.GDP.sel(Scenario="SSP2", Region=cty, Time=2021)
                                * 1e9
                                / pop
                                / 1e6
                            ],
                            y=[
                                xr_dataread.GHG_hist.sel(
                                    Time=np.arange(1950, 2021), Region=cty
                                ).sum(dim="Time")
                                / pop
                                / 1e6
                                * 1e3
                            ],
                            mode="markers",
                            # text=cty,
                            # textfont={'size': popn * 32+3, 'color': 'black'},
                            marker={
                                "size": popn * 60 + 8 * 10,
                                "color": "white",
                                "opacity": 0.5,
                                "line": {"color": "black", "width": 10},
                            },
                            showlegend=False,
                        )
                    )
                    traces_c.append(
                        go.Scatter(
                            x=[
                                xr_dataread.GDP.sel(Scenario="SSP2", Region=cty, Time=2021)
                                * 1e9
                                / pop
                                / 1e6
                            ],
                            y=[
                                xr_dataread.GHG_hist.sel(
                                    Time=np.arange(1950, 2021), Region=cty
                                ).sum(dim="Time")
                                / pop
                                / 1e6
                                * 1e3
                            ],
                            mode="markers",
                            # text=cty,
                            # textfont={'size': popn * 32+3, 'color': 'black'},
                            marker={
                                "size": popn * 60 + 8 * 2,
                                "color": cols_ctys2[cty_i],
                                "opacity": 0.9,
                                "line": {"color": "black", "width": 10},
                            },
                            showlegend=False,
                        )
                    )
    return traces_c

In [27]:
for REGION in ["EU"]:  # list(group_eu)+['EU']:
    ctys = [REGION]
    if REGION == "EU":
        ctys = group_eu

    xr_cty = xr.open_dataset(
        "K:/Data/Data_effortsharing/DataUpdate_ongoing/startyear_2021/Allocations_GHG_incl/xr_alloc_"
        + REGION
        + ".nc"
    )
    xr_med = xr_2040.sel(Temperature=1.6, Risk=0.5, **standard_settings).mean(dim="Scenario")

    # Set colouring
    cols_ctys2 = []
    for cty_i, cty in enumerate(all_countries_iso):
        xrreg = xr_med.sel(Region=cty)
        medians = np.array([xrreg[i] for i in rules_nodisc])
        idx = np.argmax(medians)
        cols_ctys2.append(rulecolors_nodisc[idx])

    traces_a = compute_traces_a(REGION, xr_cty)
    traces_b = compute_traces_b(REGION, xr_cty)
    traces_c = compute_traces_c(REGION, xr_cty)

    fig = make_subplots(
        rows=2,
        cols=3,
        specs=[[{"type": "choropleth"}, {"colspan": 2, "rowspan": 2}, {}], [{}, {}, {}]],
        horizontal_spacing=0.03,
        vertical_spacing=0.06,
    )

    # Panel a
    for t in traces_a:
        fig.add_trace(t, 1, 1)

    # Panel b
    for t in traces_b:
        fig.add_trace(t, 1, 2)

    # Panel c
    for t in traces_c:
        fig.add_trace(t, 2, 1)

    # # Other stuff
    # fig.add_scattergeo(
    #               locations = ctys,
    #               text = [ctys[r_i] for r_i in range(len(ctys))],
    #               featureidkey="properties.NAME_3",
    #               mode = 'text',
    #               hoverinfo='skip',
    #               showlegend=False,
    #               textfont={'size': 20,
    #                      'color': 'black'},
    #               row=1,
    #               col=1)
    fig.update_geos(
        visible=False,
        showlakes=True,
        lakecolor="rgb(255, 255, 255)",
        projection_type="natural earth",
        resolution=50,
        # lataxis_range=[32,70.5], lonaxis_range=[-18, 36],
        lataxis_range=[-25, 25],
        lonaxis_range=[5, 55],
        showcoastlines=False,
    )
    fig.update_layout(height=1300, width=2300)
    fig.update_geos(visible=True, showcountries=False)
    fig.update_layout(plot_bgcolor="white")
    fig.update_xaxes(row=1, col=2, range=(2010, 2050))
    fig.update_xaxes(
        row=1,
        col=2,
        ticks="outside",
        showline=True,
        linecolor="black",
        # gridcolor='lightgrey'
    )
    fig.update_yaxes(
        row=1,
        col=2,
        ticks="outside",
        showline=True,
        linecolor="black",
        # gridcolor='lightgrey'
    )
    fig.update_xaxes(
        row=2,
        col=1,
        ticks="outside",
        showline=True,
        linecolor="black",
    )
    fig.update_yaxes(
        row=2,
        col=1,
        ticks="outside",
        showline=True,
        linecolor="black",
    )
    fig["layout"].update(
        annotations=[
            dict(
                x=0.01,
                y=1.03,
                text='<b>(a) "Preferable rule" across Europe</b>',
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(color="black", size=25),
            ),
            dict(
                x=0.58,
                y=1.03,
                text="<b>(b) GHG emissions (including LULUCF) over time [Mt CO2eq/yr]",
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(color="black", size=25),
            ),
            dict(
                x=0.01,
                y=0.48,
                text="<b>(c) Wealth and historical emissions",
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(color="black", size=25),
            ),
            # dict(x=0.43, y=0.43, text='90% reduction w.r.t. 1990 emissions', xref='paper', yref='paper', showarrow=False, font=dict(color='black',  size=15)),
        ]
    )
    fig.update_xaxes(
        title_text="GDP per capita in 2021 [USD]",
        type="log",
        row=2,
        col=1,
        range=[2.7, 5],
        ticklabelposition="inside top",
    )
    fig.update_yaxes(
        title_text="GHG emissions 1950-2021 per capit<br>[kt CO<sub>2</sub>e]",
        type="log",
        row=2,
        col=1,
        range=[-1.8, 0.3],
    )
    # fig.update_yaxes(row=1, col=2, range=[np.percentile(np.array(xr_cty.sel(Time=np.arange(2021, 2051), Temperature=1.5, Risk=0.5).mean(dim='Scenario')[rules].to_dataarray()), 0.9), xr_dataread.GHG_hist.sel(Time=np.arange(1990, 2022), Region=REGION).max()])
    # fig.update_yaxes(row=1, col=2, range=[np.percentile(np.array(xr_cty.sel(Time=np.arange(2021, 2051), Temperature=1.5, Risk=0.5).mean(dim='Scenario')[rules].to_dataarray()), 0.9), 200])
    fig.update_yaxes(row=1, col=2, range=[-7e3, 6e3])
    fig.update_xaxes(tickfont=dict(size=20))
    fig.update_yaxes(tickfont=dict(size=20))
    fig.update_layout(
        legend=dict(
            yanchor="bottom",
            y=0.01,
            xanchor="left",
            x=0.35,
            font=dict(size=20, color="black"),
        )
    )
    fig.write_image("../Figures/Paper_Europe/Focus_" + REGION + ".png", scale=5)
    3

In [14]:
# fig = make_subplots(rows=1, cols=2,
#                     subplot_titles=["<b>Allocations in 2030<br>(absolute)", "<b>Emission reductions<br>(2030 compared to 2015)"],
#                     horizontal_spacing = 0.07, vertical_spacing=0.06)
# # 2030 allocations
# fig.add_traces(go.Scatter(x=['NDC'],
#                         y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines+markers',
#                         marker={'size': 17, 'symbol': 'triangle-down'},
#                         showlegend=False), 1, 1)
# fig.add_traces(go.Scatter(x=['NDC'],
#                         y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines+markers',
#                         marker={'size': 17, 'symbol': 'triangle-up'},
#                         showlegend=False), 1, 1)
# fig.add_traces(go.Scatter(x=['NDC', 'NDC'],
#                         y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min(),
#                         xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines',
#                         marker={'size': 11, 'symbol': 'triangle-up'},
#                         showlegend=False), 1, 1)
# try:
#     fig.add_traces(go.Scatter(x=['Current<br>Policies']*len(xr_eng.Model),
#                             y=xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value,
#                             name='Current policies',
#                             mode='lines+markers',
#                             marker_color='silver',
#                             marker_line_color='black',
#                             marker_line_width=1,
#                             marker={'size': 9, 'symbol': 'square'},
#                             showlegend=False),
#                     1, 1
#                     )
#     vals = np.array(xr_scen_r.sel(Time=2030, Region=REG, Variable="Emissions|CO2").Value)
#     fig.add_traces(go.Violin(x=["Cost<br>optimal"]*len(vals),
#                                     y=vals,
#                                     showlegend=False,
#                                     points=False,
#                                     opacity=1.0,
#                                     line={'color': 'tomato', 'width': 3},
#                                     line_color='tomato'),
#                             1, 1
#                             )
#     vals2 = np.array(xr_scen_r2.sel(Time=2030, Region=REG, Variable="Emissions|CO2").Value)
#     fig.add_traces(go.Violin(x=["Cost<br>optimal"]*len(vals2),
#                                     y=vals2,
#                                     showlegend=False,
#                                     points=False,
#                                     opacity=0.5,
#                                     line={'color': 'tomato', 'width': 3},
#                                     line_color='tomato'),
#                             1, 1
#                             )
#     med = np.nanmedian(vals)
#     fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies', "Cost<br>optimal"],
#                                     y=[np.nanmedian(vals)]*3,
#                                     line={'color': 'tomato', 'width': 2, 'dash': 'dot'},
#                                     mode='lines',
#                                     showlegend=False), 1, 1
#                     )
#     fig.add_traces(go.Scatter(x=['NDC'],
#                                     y=[np.nanmedian(vals)],
#                                     marker={'color': 'tomato', 'size': 15},
#                                     mode='markers',
#                                     showlegend=False), 1, 1
#                     )
#     regval = float(xr_dataread.sel(Region=REG, Time=2015).CO2_hist)
#     fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies']+list(rules[:rule_i+1]),
#                             y=[regval]*(len(rules[:rule_i+1])+2),
#                             line={'color': 'black', 'width': 1, 'dash': 'dash'},
#                             mode='lines',
#                             showlegend=False), 1, 1
#             )
#     fig.add_traces(go.Scatter(x=['NDC'],
#                             y=[regval],
#                             text='         2015<br>     emissions',
#                             line={'color': 'black', 'width': 2, 'dash': 'dash'},
#                             textfont={'color': 'black', 'size': 14},
#                             textposition='middle right',
#                             mode='text',
#                             showlegend=False), 1, 1
#             )
# except:
#     3
# # fig.add_traces(go.Scatter(x=['NDC', 'CurPol'],
# #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())]*2,
# #                         line={'color': 'silver', 'width': 2, 'dash': 'dot'},
# #                         mode='lines',
# #                         showlegend=False),
# #                 1, 1
# #                 )
# # fig.add_traces(go.Scatter(x=['NDC'],
# #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())],
# #                         marker={'color': "grey", 'size': 15},
# #                         mode='markers',
# #                         showlegend=False), 1, 1
# #                 )
# for rule_i, rule in enumerate(rules):
#     for T_i, T in enumerate([1.5, 2.0]):
#         vals = np.array(xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=2030)).flatten()
#         vals[vals == 0] = np.nan
#         if T == 1.5:
#             med = np.nanmedian(vals)
#             fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies', 'Cost<br>optimal']+list(rules[:rule_i+1]),
#                                 y=[np.nanmedian(vals)]*(len(rules[:rule_i+1])+3),
#                                 line={'color': rulecolors[rule_i], 'width': 1, 'dash': 'dot'},
#                                 mode='lines',
#                                 showlegend=False), 1, 1
#                     )
#             fig.add_traces(go.Scatter(x=['NDC'],
#                                 y=[np.nanmedian(vals)],
#                                 marker={'color': rulecolors[rule_i], 'size': 15},
#                                 mode='markers',
#                                 showlegend=False), 1, 1
#                     )
#         fig.add_traces(go.Violin(x=[rule]*len(vals),
#                                 y=vals,
#                                 showlegend=False,
#                                 points=False,
#                                 opacity=1.0-T_i*0.5,
#                                 line={'color': 'black', 'width': 3},
#                                 line_color=rulecolors[rule_i]),
#                         1, 1
#                         )
# fig.add_traces(go.Scatter(x=['NDC'],
#                         y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines+markers',
#                         marker={'size': 17, 'symbol': 'triangle-down'},
#                         showlegend=False), 1, 1)
# fig.add_traces(go.Scatter(x=['NDC'],
#                         y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines+markers',
#                         marker={'size': 17, 'symbol': 'triangle-up'},
#                         showlegend=False), 1, 1)

# # Reductions in 2030 w.r.t. 2021
# regval = float(xr_dataread.sel(Region=REG, Time=2015).CO2_hist)
# fig.add_traces(go.Scatter(x=['NDC'],
#                         y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines+markers',
#                         marker={'size': 17, 'symbol': 'triangle-up'},
#                         showlegend=False),  1, 2)
# fig.add_traces(go.Scatter(x=['NDC'],
#                         y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines+markers',
#                         marker={'size': 17, 'symbol': 'triangle-down'},
#                         showlegend=False),  1, 2)
# fig.add_traces(go.Scatter(x=['NDC', 'NDC'],
#                         y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100,
#                         (xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines',
#                         marker={'size': 11, 'symbol': 'triangle-down'},
#                         showlegend=False),  1, 2)
# try:
#     fig.add_traces(go.Scatter(x=['Current<br>Policies']*len(xr_eng.Model),
#                             y=(np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value) - regval)/regval*-100,
#                             name='Current policies',
#                             mode='lines+markers',
#                             marker_color='silver',
#                             marker_line_color='black',
#                             marker_line_width=1,
#                             marker={'size': 9, 'symbol': 'square'},
#                             showlegend=False),
#                     1, 2
#                     )
#     vals = (np.array(xr_scen_r.sel(Time=2030, Region=REG, Variable="Emissions|CO2").Value) - regval)/regval*-100
#     fig.add_traces(go.Violin(x=["Cost<br>optimal"]*len(vals),
#                                     y=vals,
#                                     showlegend=False,
#                                     points=False,
#                                     opacity=1.0,
#                                     line={'color': 'tomato', 'width': 3},
#                                     line_color='tomato'),
#                             1, 2
#                             )
#     vals2 = (np.array(xr_scen_r2.sel(Time=2030, Region=REG, Variable="Emissions|CO2").Value) - regval)/regval*-100
#     fig.add_traces(go.Violin(x=["Cost<br>optimal"]*len(vals2),
#                                     y=vals2,
#                                     showlegend=False,
#                                     points=False,
#                                     opacity=0.5,
#                                     line={'color': 'tomato', 'width': 3},
#                                     line_color='tomato'),
#                             1, 2
#                             )
#     med = np.nanmedian(vals)
#     fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies', "Cost<br>optimal"],
#                                     y=[np.nanmedian(vals)]*3,
#                                     line={'color': 'tomato', 'width': 1, 'dash': 'dot'},
#                                     mode='lines',
#                                     showlegend=False), 1, 2
#                     )
#     fig.add_traces(go.Scatter(x=['NDC'],
#                                     y=[np.nanmedian(vals)],
#                                     marker={'color': 'tomato', 'size': 15},
#                                     mode='markers',
#                                     showlegend=False), 1, 2
#                     )
#     fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies']+list(rules[:rule_i+1]),
#                             y=[0]*(len(rules[:rule_i+1])+2),
#                             line={'color': 'black', 'width': 2, 'dash': 'dash'},
#                             mode='lines',
#                             showlegend=False), 1, 2
#             )
# except:
#     3
# # fig.add_traces(go.Scatter(x=['NDC', 'CurPol'],
# #                         y=[(np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median()) - regval)/regval*-100]*2,
# #                         line={'color': 'silver', 'width': 2, 'dash': 'dot'},
# #                         mode='lines',
# #                         showlegend=False),
# #                 1, 2
# #                 )
# # fig.add_traces(go.Scatter(x=['NDC'],
# #                         y=[(np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median()) - regval)/regval*-100],
# #                         marker={'color': "grey", 'size': 15},
# #                         mode='markers',
# #                         showlegend=False), 1, 2
# #                 )
# for rule_i, rule in enumerate(rules):
#     for T_i, T in enumerate([1.5, 2.0]):
#         vals = (np.array(xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=2030)).flatten()-regval)/regval*-100
#         vals[vals == 0] = np.nan
#         if T == 1.5:
#             med = np.nanmedian(vals)
#             fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies', 'Cost<br>optimal']+list(rules[:rule_i+1]),
#                                 y=[np.nanmedian(vals)]*(len(rules[:rule_i+1])+3),
#                                 line={'color': rulecolors[rule_i], 'width': 1, 'dash': 'dot'},
#                                 mode='lines',
#                                 showlegend=False), 1, 2
#                     )
#             fig.add_traces(go.Scatter(x=['NDC'],
#                                 y=[np.nanmedian(vals)],
#                                 marker={'color': rulecolors[rule_i], 'size': 15},
#                                 mode='markers',
#                                 showlegend=False), 1, 2
#                     )
#         fig.add_traces(go.Violin(x=[rule]*len(vals),
#                                 y=vals,
#                                 showlegend=False,
#                                 points=False,
#                                 opacity=1.0-T_i*0.5,
#                                 line={'color': 'black', 'width': 3},
#                                 line_color=rulecolors[rule_i]),
#                         1, 2
#                         )
# fig.add_traces(go.Scatter(x=['NDC'],
#                         y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines+markers',
#                         marker={'size': 17, 'symbol': 'triangle-up'},
#                         showlegend=False),  1, 2)
# fig.add_traces(go.Scatter(x=['NDC'],
#                         y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
#                         name='NDCs',
#                         line={'color': 'black', 'width': 3},
#                         mode='lines+markers',
#                         marker={'size': 17, 'symbol': 'triangle-down'},
#                         showlegend=False),  1, 2)
# fig.update_annotations(font=dict(family="Helvetica", size=25))
# fig.update_layout(height=800, width=1900, template='simple_white', font=dict(family="Helvetica"))
# fig.update_yaxes(tickfont=dict(size=20), row=1, col=1, title='CO<sub>2</sub> emissions allocations in 2030 ('+REG+')<br>[Mt CO<sub>2</sub>]', titlefont=(dict(size=20)))
# fig.update_xaxes(tickfont=dict(size=20), row=1, col=1)
# fig.update_yaxes(tickfont=dict(size=20), row=1, col=2, title='% reduction with respect to 2015', titlefont=(dict(size=20)), range=(-150, 150), tickvals=[-150, -100, -50, 0, 50, 100, 150], ticktext=['-150%', '-100%', '-50%', '0%', '50%', '100%', '150%'])
# fig.update_xaxes(tickfont=dict(size=20), row=1, col=2)
# #     fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Alloc2030_"+REG+".png", scale=3)
# # fig.show()